In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**things to check about this data**
* number of cancelation - to understand what will be a improve

**cluster the No show**
* which day there is the most cancellation 
* which hour in the days
* what age and gender
* Neighborhood with more cancelation - maybe heat map... 
* check if persons with handicap is more No show, maybe we can help them  

**suggestion** 
* what is the best time to make an appointment - recheck about this to minimize cancelation 
* check the value of the remainder of SMS 



In [ ]:

#arrange the data 
filename = "/kaggle/input/noshowappointments/KaggleV2-May-2016.csv"
df = pd.read_csv(filename)


In [ ]:
from  datetime import  datetime
import calendar



#add more columns for the analysis     
df["ScheduledDay"] =pd.to_datetime(df["ScheduledDay"])
df["AppointmentDay"] =pd.to_datetime(df["AppointmentDay"])
df["ScheduledHour"] = pd.to_datetime(df["ScheduledDay"]).apply(lambda p:p.hour)
df["Scheduledmin"] = pd.to_datetime(df["ScheduledDay"]).apply(lambda p:p.minute)
df["ScheduledDay_word"] = pd.to_datetime(df["ScheduledDay"]).apply(lambda p:calendar.day_name[p.weekday()])
df["ScheduledDay_num"] = pd.to_datetime(df["ScheduledDay"]).apply(lambda p:p.weekday)
df["between_reg2app"] = df.AppointmentDay.dt.date - df.ScheduledDay.dt.date


df.head(10)

In [ ]:
df.describe() # looking if there is problem with the data

we can notice there is a problem with the data, there is at least one row with age <0.

In [ ]:
df.info()

In [ ]:
#find the Eror and remove them 
x =df[df["Age"]<0].index
df.drop(x,inplace=True)

#check if the problem solved
df.describe()

In [ ]:
df.info()

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

#arange the data for the plot
M_num_show = len(df[(df["No-show"]=="No")&(df["Gender"]=="M")])
F_num_show = len(df[(df["No-show"]=="No")&(df["Gender"]=="F")])
F_num_nshow = len(df[(df["No-show"]=="Yes")&(df["Gender"]=="F")])
M_num_nshow = len(df[(df["No-show"]=="Yes")&(df["Gender"]=="M")])

y_men = [M_num_show,M_num_nshow,]
y_women= [F_num_show,F_num_nshow]

#arange general settings for the plot
x = np.arange(len(y_men))
width = 0.4
color_p = ["orange","orange"]
labels = ["Show","No-Show"]

#setting of the plot
fig, ax = plt.subplots()
plt.style.use("ggplot") 
W_no=ax.bar(x-0.2,  y_women,width= 0.4,color="orange",label='Women')
M_no=ax.bar(x+0.2, y_men,width=0.4,color = "blue",label='Men')
ax.set_xticks(x, labels)
ax.set_title('Show VS No-show by gender')
ax.bar_label(W_no,padding=1)
ax.bar_label(M_no, padding=1)
ax.legend()


#for the review 
total_paintes = len(df)
numb_noshow = len(df[(df["No-show"]=="Yes")])
print("% of No-show painets: ", str(round(numb_noshow/total_paintes,3)))
print("% of Men No-show painets : " , str(round(M_num_nshow/(M_num_nshow+M_num_show),3)))
print("% of Women No-show painets : " , str(round(F_num_nshow/(F_num_nshow+F_num_show),3)))


#another shot to do this with fewer lines - for the learing proccess 
#a= df.groupby(["No-show","Gender"]).count()["PatientId"]
#print(a.unstack().plot(kind="bar",color = ["orange","blue"]))

**there are some interesting insights here **
* there is a lot of patines who doesn't come (0.202%), and if we will know the price of the medical staff we can calculate the percentage of cancellation we aim to achieve
* there is no big difference between Women and Men (Men-0.2%, Women- 02.03%)

In [ ]:
### #which day the most didnt come
y = df.copy() 
y["No-show"].replace({0:"Women",1:"Men"},inplace=True)

y1= y.groupby(["ScheduledDay_num",'No-show']).count().sort_index()

y= y1["PatientId"].unstack()
y= y["Yes"]/(y["No"]+y["Yes"])*100




y[6]=0
x=np.arange(7)

fig, ax = plt.subplots()
plt.style.use("ggplot") 

prece =ax.bar(x, y,color = "blue")


#total = ax.bar(x, y1,color = "green")
week_day =  ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
plt.xticks(np.arange(len(week_day )),week_day ,rotation= 45)
plt.title('% No-show in days of the week')



In [ ]:
fig, ax = plt.subplots()
plt.style.use("ggplot") 
x = np.arange(7)
y = df.groupby("ScheduledDay_num").count()["PatientId"]
y2 = df[df["No-show"]=="Yes"].groupby("ScheduledDay_num").count()["PatientId"]
y[6]=0
y2[6]=0

print(x)
tot =  ax.bar(x, y,color = "green")
no = ax.bar(x, y2,color = "blue")


week_day =  ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
plt.xticks(np.arange(len(week_day )),week_day ,rotation= 45)
ax.bar_label(tot,padding=1)
ax.bar_label(no,padding=1,color="w")


y.head(10)

In [ ]:
y = df.copy() 
y["No-show"].replace({0:"Women",1:"Men"},inplace=True)

y1= y.groupby(["ScheduledHour",'No-show']).count().sort_index()
total =  y.groupby(["ScheduledHour"]).count().sort_index()["PatientId"]


y= y1["PatientId"].unstack()
y["prece"]= y["Yes"]/(y["No"]+y["Yes"])*100


fig, ax = plt.subplots(2,1)

ax[0].plot(y["prece"],color="blue")
ax[0].set_xlabel('Time')
ax[0].set_ylabel('% of cancelation', color='blue')
#ax2 = ax.twinx()
ax[1].plot(total,color='red')
ax[1].set_xlabel('Time')
ax[1].set_ylabel('total appoitments', color='red')
plt.show()


as the appoitment is at the morning it has less probability to no-show, mean no show while the day is 22.55, the lowest is between 7 to 8 and 15.15 and after 15:00 we aboue the mean and we get into 30 and 33.3 precent at 20-22

another thing is that  we have a high decrase in the numb of the appoitment, the mean appoitment for hour is 6907 after 17:00 we have 2,909, 1349,488,100,3

sugestion - bring more medical staff into the morning btween 7-10 and close the option to set appoitment after 18 or bring less medical staff



we can see that there is no big 

In [ ]:
#precente of handcap that No-Show
y=df[df["Handcap"]>0]
y_no = y[y["No-show"]=="Yes"].count()["PatientId"]
round(y_no/len(y),2)
y.head()

#same patterns as un-handicap persons in days and hours 
x1= y.groupby(["ScheduledDay_num"]).count()["PatientId"]
x1.head(100)
x2= y.groupby(["ScheduledHour"]).count()["PatientId"]
x2.head(100)


#fig, ax = plt.subplots()
#ax.plot(x=y.index,y=y[])

#y.plot(kind="bar")


is this chapster we can se the handicap has the same patterns as the other one...

In [ ]:

def age_dec(x):
    if len(str(x))==1:
        return 0
    return int(str(x)[0])

df["Age_deceade"] = df["Age"].apply(age_dec)

x= df.groupby(["Age_deceade","No-show"]).count()
y=x.unstack()["PatientId"]

precenet =y["Yes"]/(y["Yes"]+y["No"])
precenet.plot(kind="line")
print()

x= df.groupby(["Age_deceade"]).count()["PatientId"]

print(x)
z=df[df["No-show"]=="Yes"]["Age_deceade"]

print(precenet)

print(x*precenet)

the % of no show is 19.5 close to all the population, 
 there is 3 groups which has more precentage 10-20 (25.1), 20-30 (24.6), adding to this that those age range is the second and the forth in the amount of appoitments is giving the sugestoin that we need to call those persoans first.. 

In [ ]:
###### chek the hist about the time and from setting the appoitment and the cancelation 

y= df.groupby(["between_reg2app","No-show"]).count()["PatientId"].unstack()

y=y.iloc[2:24,:]
y["total"] =  y["Yes"] + y["No"]
precent =  y["Yes"]/y["total"]


x1 = np.arange(22)
x_label = [str(item)[:6] for item in precent.index ]
x_label =[x_label[i] for i in range(len(x_label)) if i%2==0] 
ind = np.arange(0,22,2)

fig, ax = plt.subplots()
ax.plot(x1,precent ,color="blue")
ax.set_xticks(ind,x_label,rotation= 45)

print(precent)

after 10 days it is the moment the precentege go up to 30 precent, and 7 days before it is  0.266816
 so if we will call them 7 days before and "set the appoitment again" we will improve the chances there will show up

In [ ]:
#check the the precent in each niebherhood

df.head()
y  = df.groupby(["Neighbourhood","No-show"]).count()["PatientId"]
y1 = y.unstack()

y1["total"] = y1["No"]+y1["Yes"]


y1["prec"] = y1["Yes"]/y1["total"]  
y1["grade"] =(y1["prec"]-y1["prec"].min())/y1["prec"].std() 
y1.head(30)

#conc there is not a unique niberhood such that a lot of pepole didnt come
# i dont undersatnd whey the st_grade is not around 0 
#ax.plot(kind="bar",x=  )

In [ ]:
#the affect of SMS about the cancetltion 
df.head()
get_sms = df[df["SMS_received"]==1]
no_sms =  df[df["SMS_received"]==0]

prec_cancel_get_sms= len(get_sms[get_sms["No-show"]=="Yes"]["PatientId"])/len(get_sms["PatientId"])
prec_cancel_get_sms

prec_cancel_no_sms= len(no_sms[no_sms["No-show"]=="Yes"]["PatientId"])/len(no_sms["PatientId"])
prec_cancel_no_sms

x1=[1,2]
y2= [round(prec_cancel_get_sms,2),round(prec_cancel_no_sms,2)]
label = ["No-SMS","SMS"]

fig, ax = plt.subplots()
plt.style.use("ggplot")
dat = ax.bar(x1,y2,color = ["blue","green"])
ax.bar_label(dat,padding=1)
ax.set_title('SMS VS No-SMS % of cancel')
ax.set_xticks(x1, label)
ax.legend()
 


In [ ]:
#how many appoitments at each day 
df[]

In [ ]:

A#check first how many does not get in time b
num_no_show = NoShow_Df[NoShow_Df["No-show"]=="No"].count()
print(num_no_show)
num_show = NoShow_Df[NoShow_Df["No-show"]=="Yes"].count()
data = [num_show,num_no_show]


In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline


num_no_show = NoShow_Df[NoShow_Df["No-show"]==1].count()["No-show"]
num_show = NoShow_Df[NoShow_Df["No-show"]==0].count()["No-show"]
data = [num_show,num_no_show]
print(num_no_show,num_show)


fig = plt.figure(figsize =(5, 3))
plt.pie(data, labels = ["Arrived on Date","Not Arrived at all"],colors = ["orange","green"],explode = (0.1,0.0),startangle=110,autopct='%1.1f%%')
plt.title("Arrived VS Not arrived patients")


In [ ]:
#next q is to ask what if the ratio between wemoan and men 
#change gender into more sutiable values, F=0 and M=1 
NoShow_Df["Gender"].replace({"F":0,"M":1},inplace=True)
numb_men = NoShow_Df[NoShow_Df["Gender"]==1].count()["Gender"]
numb_wemoan =  NoShow_Df[NoShow_Df["Gender"]==0].count()["Gender"]
fig = plt.figure(figsize =(3,3))
plt.bar([1],[numb_men],label= "Men", width= 0.6)
plt.bar([2],[numb_wemoan],label="Men",width= 0.6)
plt.xticks([1,2], ["Men","Women"])
plt.title('Men VS Women no-Show')


plt.legend()

plt.show()

In [ ]:
#which day the most didnt come
y = NoShow_Df.copy() 
y["No-show"].replace({0:"Women",1:"Men"},inplace=True)

y1= y.groupby(["ScheduledDay_num",'No-show']).count().sort_index()

y1["PatientId"].unstack().plot(kind="bar",stacked=False,grid=True)

week_day =  ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
plt.xticks(np.arange(len(week_day )),week_day )
plt.title('No show in days of the week')


In [ ]:
import datetime
#duration between registration and appointment 
y= NoShow_Df.drop(NoShow_Df[NoShow_Df["between_reg2app"]<"0 days"].index,inplace=True)
y= NoShow_Df.groupby(["between_reg2app","No-show"]).count()



#dt_date.strftime("%A, %d %b %Y")
y=y["PatientId"].unstack()
print(y.head())
fig,ax = plt.subplots()
range_time = np.arange(len(y.index))



#y.index
#range_time= []
#for i in y.index:
 #   a=i.strftime("%%")
 #   range_time.append(a)
#print(range_time)
#ax.plot(x=y.index,y=y[0])
#y.head(400)